# Downloading Dataset

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: cuntwars_user_id=vFHd55haQ6; _ga=GA1.3.1069537971.1546461315; __utma=68291539.1069537971.1546461315.1547285225.1547285225.1; __utmz=68291539.1547285225.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none)" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-datasets/122892/296485/celebrities-100k.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1560182347&Signature=bsOOojV8YEBXx8yeRQIH2%2BFXuS1Wn%2BMttvFgeUVD17YqUuakDVfeZCDCITaqOBYQ2mpVOh1ypeRb7MS325pGnxRIEXbf9QkMKbGyufKeyU91jilDh80xmjc%2F6Rte%2B4RA3j1sOER7iahP68LQzhNFph8uNLPA%2FolyORKeiteOYZ7IMRanLbnAcgzDePfbnQGFlnz7ybQMySXILkeKBSV%2F6nwjuHosPuUZKqKIELZ1dsGTau0b7Clo3iqLm7wPS2zYldxk9%2FttLvPtbZG56xX7zFNqwWTmW4GsJ2mLzj6CdeisQTo4gh4mhhenlQ2dnxA0YWdeK3vOzEyP%2Fpo0cfP8UQ%3D%3D" -O "celebrities-100k.zip" -c

--2019-06-07 15:59:21--  https://storage.googleapis.com/kaggle-datasets/122892/296485/celebrities-100k.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1560182347&Signature=bsOOojV8YEBXx8yeRQIH2%2BFXuS1Wn%2BMttvFgeUVD17YqUuakDVfeZCDCITaqOBYQ2mpVOh1ypeRb7MS325pGnxRIEXbf9QkMKbGyufKeyU91jilDh80xmjc%2F6Rte%2B4RA3j1sOER7iahP68LQzhNFph8uNLPA%2FolyORKeiteOYZ7IMRanLbnAcgzDePfbnQGFlnz7ybQMySXILkeKBSV%2F6nwjuHosPuUZKqKIELZ1dsGTau0b7Clo3iqLm7wPS2zYldxk9%2FttLvPtbZG56xX7zFNqwWTmW4GsJ2mLzj6CdeisQTo4gh4mhhenlQ2dnxA0YWdeK3vOzEyP%2Fpo0cfP8UQ%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 860082689 (820M) [application/zip]
Saving to: ‘celebrities-100k.zip’

celebrities-100k.zi 100%[===================>] 820.24M  77.1MB/s    in 11s     

2019-06-07 15:59:32 (75.6 MB/s) - ‘

In [0]:
!unzip 'celebrities-100k.zip'

Archive:  celebrities-100k.zip
  inflating: 100k.zip                
  inflating: 100k.txt                


In [0]:
!unzip '100k.zip'

# Building Model and Training

In [0]:
!mkdir 'Model Architecture transposed_convolutions'

In [2]:
from __future__ import print_function, division

from keras.layers import Dense, Conv2DTranspose, Dropout, Flatten, Reshape, Input, BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D, AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from PIL import Image
from keras.utils import plot_model
from numpy import random

import matplotlib.pyplot as plt
import numpy as np
import sys
import os as os
import tensorflow as tf

Using TensorFlow backend.


In [0]:
noise_shape = 100
nh = 128
nw = 128
nc = 3
img_shape = (nh, nw, nc)

# Carefully chosen parameters 
d_opt = Adam(lr = 0.00004, beta_1 = 0.5)
g_opt = Adam(lr = 0.0002, beta_1 = 0.5)

WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005

In [0]:
def build_generator():
  
  # 8x8x1024
  model = Sequential()
  model.add(Dense(1024 * 8 * 8, input_dim = noise_shape, name = 'Dense_Layer'))
  model.add(Reshape((8, 8, 1024) , name = "Reshape_Layer"))
  model.add(LeakyReLU(name = 'Leaky_ReLU'))

  # 8x8x1024 -> 16x16x512
  model.add(Conv2DTranspose(512, kernel_size = 5, strides = 2, padding ="same", kernel_initializer = tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "Transposed_Convolution"))  
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_2'))

  # 16x16x512 -> 32x32x256
  model.add(Conv2DTranspose(256, kernel_size = 5, strides = 2, padding ="same", kernel_initializer = tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "Transposed_Convolution_2"))  
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_2"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_3'))
  
  # 32x32x256 -> 64x64x128
  model.add(Conv2DTranspose(128, kernel_size = 5, strides = 2, padding ="same", kernel_initializer = tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "Transposed_Convolution_3"))  
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_3"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_4'))
  
  # 64x64x128 -> 128x128x64
  model.add(Conv2DTranspose(64, kernel_size = 5, strides = 2, padding ="same", kernel_initializer = tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "Transposed_Convolution_4"))  
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_4"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_5'))
  
  # 128x128x64 -> 128x128x3
  model.add(Conv2DTranspose(3, kernel_size = 5, strides = 1, padding ="same", kernel_initializer = tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "Transposed_Convolution_5"))  
  
  #TANH ACTIVATION
  model.add(Activation("tanh"))
    
  print(model.summary())
  
  #Picturizing model
  plot_model(model, to_file='Model Architecture transposed_convolutions/generator.png')
  
  noise = Input(shape = (noise_shape,))
  img = model(noise)
  
  return Model(inputs = noise, outputs = img, name = "Generator")
  

In [5]:
generator = build_generator()
generator.summary()

W0614 14:38:19.051524 140680200431488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0614 14:38:19.054923 140680200431488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0614 14:38:19.059042 140680200431488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0614 14:38:19.127402 140680200431488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0614 14:38:19.128325 1406802004

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_Layer (Dense)          (None, 65536)             6619136   
_________________________________________________________________
Reshape_Layer (Reshape)      (None, 8, 8, 1024)        0         
_________________________________________________________________
Leaky_ReLU (LeakyReLU)       (None, 8, 8, 1024)        0         
_________________________________________________________________
Transposed_Convolution (Conv (None, 16, 16, 512)       13107712  
_________________________________________________________________
Batch_Norm (BatchNormalizati (None, 16, 16, 512)       2048      
_________________________________________________________________
Leaky_ReLU_2 (LeakyReLU)     (None, 16, 16, 512)       0         
_________________________________________________________________
Transposed_Convolution_2 (Co (None, 32, 32, 256)       3277056   
__________

In [0]:
def build_discriminator():
  model = Sequential()
  
  # 128*128*3 -> 64x64x64
  model.add(Conv2D(64, kernel_size=5, strides = 2, input_shape = img_shape, padding="same", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = 'Conv_Layer'))
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm"))
  model.add(LeakyReLU(name = 'Leaky_ReLU'))
  
  # 64x64x64-> 32x32x128
  model.add(Conv2D(128, kernel_size=5, strides = 2, padding="same", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = 'Conv_Layer_2'))
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_2"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_2'))
  
  # 32x32x128 -> 16x16x256
  model.add(Conv2D(256, kernel_size=5, strides = 2, padding="same", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = 'Conv_Layer_3'))
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_3"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_3'))
  
  # 16x16x256 -> 16x16x512
  model.add(Conv2D(512, kernel_size=5, strides = 1, padding="same", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = 'Conv_Layer_4'))
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_4"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_4'))
  
  # 16x16x512 -> 8x8x1024
  model.add(Conv2D(1024, kernel_size=5, strides = 2, padding="same", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = 'Conv_Layer_5'))
  model.add(BatchNormalization(epsilon = EPSILON, name = "Batch_Norm_5"))
  model.add(LeakyReLU(name = 'Leaky_ReLU_5'))
  
  model.add(Flatten())
  model.add(Dense(1, activation = None))
  model.add(Activation("sigmoid"))
  
  print(model.summary())
  
  #Picturizing model
  plot_model(model, to_file='Model Architecture transposed_convolutions/discriminator.png')
  
  img = Input(shape = img_shape)
  validity = model(img)
  
  return Model(inputs = img, outputs = validity, name = 'Discriminator')

In [7]:
discriminator = build_discriminator()
discriminator.summary()

discriminator.compile(loss = 'binary_crossentropy', optimizer = d_opt, metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_Layer (Conv2D)          (None, 64, 64, 64)        4864      
_________________________________________________________________
Batch_Norm (BatchNormalizati (None, 64, 64, 64)        256       
_________________________________________________________________
Leaky_ReLU (LeakyReLU)       (None, 64, 64, 64)        0         
_________________________________________________________________
Conv_Layer_2 (Conv2D)        (None, 32, 32, 128)       204928    
_________________________________________________________________
Batch_Norm_2 (BatchNormaliza (None, 32, 32, 128)       512       
_________________________________________________________________
Leaky_ReLU_2 (LeakyReLU)     (None, 32, 32, 128)       0         
_________________________________________________________________
Conv_Layer_3 (Conv2D)        (None, 16, 16, 256)       819456    
__________

W0614 14:38:24.124400 140680200431488 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0614 14:38:24.135329 140680200431488 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 17488257  
Total params: 17,488,257
Trainable params: 17,484,289
Non-trainable params: 3,968
_________________________________________________________________


In [8]:
z = Input(shape = (noise_shape,), name = "Input_Noise")
img = generator(z)

discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss = 'binary_crossentropy', optimizer = g_opt)
combined.summary()

#Picturizing model
plot_model(combined, to_file='Model Architecture transposed_convolutions/combined.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Noise (InputLayer)     (None, 100)               0         
_________________________________________________________________
Generator (Model)            (None, 128, 128, 3)       24036739  
_________________________________________________________________
Discriminator (Model)        (None, 1)                 17488257  
Total params: 41,524,996
Trainable params: 24,034,819
Non-trainable params: 17,490,177
_________________________________________________________________


In [9]:
print(generator.summary())
print(discriminator.summary())
print(combined.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 128, 128, 3)       24036739  
Total params: 24,036,739
Trainable params: 24,034,819
Non-trainable params: 1,920
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 17488257  
Total params: 34,972,546
Trainable params: 17,484,289
Non-trainable params: 17,488,257
_________________________________________________________________
None
_____________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [0]:
def get_batch(width, height):
  image_files = os.listdir('/content/100k/')[:40000]
  data_batch = []
  
  for file in image_files:
    img = Image.open('/content/100k/'+file).resize([width, height])
    data_batch.append(np.array(img.convert('RGB')))
    
  data_batch = np.array(data_batch)
  print(data_batch.shape)
  return data_batch

In [0]:
def train(batch_size = 128, epochs = 15830):
  
  
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))
  
  X_train = get_batch(nw, nh)
  X_train = (X_train.astype(np.float32) / 127.5) - 1.0
  
  #adding noise to input images
  for i in range(len(X_train)):
    X_train[i] = X_train[i] + np.random.normal(size = X_train[i].shape, loc = 0.0, scale = random.uniform(0.0, 0.1)).astype(np.float32)
  
  for epoch in range(70000, epochs):
    
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, noise_shape))
    
    gen_imgs = generator.predict(noise)

    #Training Discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid*random.uniform(0.9, 1.0))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5*np.add(d_loss_real, d_loss_fake)

    #Training Generator
    g_loss = combined.train_on_batch(noise, valid)

    #Progress
    if (epoch+1)%50 == 0:
      print("epoch: " + str(epoch+1) + " " + "D_Loss = " + str(d_loss[0]) + " " + "acc: " + str(d_loss[1]*100) + " " +  "G_Loss = " + str(g_loss))
    
    if (epoch+1)%10000 == 0 or epoch+1 == 1000:
      save_imgs(epoch+1)
      generator.save_weights('generator.h5')
      discriminator.save_weights('discriminator.h5')

In [0]:
!mkdir 'images_transposed_convolutions'

In [0]:
def save_imgs(epoch):
  r, c = 3, 3
  noise = np.random.normal(0, 1, (r * c, noise_shape))
  gen_imgs = generator.predict(noise)

  # Rescale images 0 - 1
  gen_imgs = (1/2.5) * gen_imgs + 0.5
  
  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,:])
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig("images_transposed_convolutions/%d.png" % epoch)
  plt.close()

In [0]:
#generator.load_weights('generator.h5')
#discriminator.load_weights('discriminator.h5')

In [0]:
train(64, 100000)

(40000, 128, 128, 3)
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 70050 D_Loss = 0.10552743 acc: 50.0 G_Loss = 1.0561694
epoch: 70100 D_Loss = 0.04420022 acc: 50.0 G_Loss = 0.20355316
epoch: 70150 D_Loss = 0.06738627 acc: 50.0 G_Loss = 0.09743933
epoch: 70200 D_Loss = 0.044008028 acc: 50.0 G_Loss = 0.17431289
epoch: 70250 D_Loss = 0.14734924 acc: 50.0 G_Loss = 0.2036498
epoch: 70300 D_Loss = 0.21408778 acc: 50.0 G_Loss = 1.7051105
epoch: 70350 D_Loss = 0.07258231 acc: 50.0 G_Loss = 0.02706325
epoch: 70400 D_Loss = 0.111648515 acc: 50.0 G_Loss = 0.024665382
epoch: 70450 D_Loss = 0.052209817 acc: 50.0 G_Loss = 0.036941327
epoch: 70500 D_Loss = 0.08552354 acc: 50.0 G_Loss = 0.027282452
epoch: 70550 D_Loss = 0.11972844 acc: 50.0 G_Loss = 0.28563154
epoch: 70600 D_Loss = 0.15278652 acc: 50.0 G_Loss = 0.20852631
epoch: 70650 D_Loss = 0.072271794 acc: 50.0 G_Loss = 0.21463333
epoch: 70700 D_Loss = 0.17997192 acc: 50.0 G_Loss = 0.9117613
epoch: 70750 D_Loss = 0.11500873 acc: 50.0 G_Loss = 0.33619756
epoch: 70800 D_Loss = 0.1691483 acc: 50.0 G_Loss = 0

In [0]:
generator.save_weights('generator.h5')
discriminator.save_weights('discriminator.h5')

# Generating Images

In [0]:
save_imgs(100000)

In [0]:
noise = np.random.normal(0, 1, (1, noise_shape))
gen_imgs = generator.predict(noise)
gen_imgs = gen_imgs*(1/2.5) + 0.5

plt.imshow(gen_imgs.reshape((128 , 128, 3)))

In [0]:
generator.save_weights('generator.h5')
discriminator.save_weights('discriminator.h5')